In [1]:
import textract as tx
import pandas as pd
import os

In [2]:
def check_if_title(sentence):
    sentence = str(sentence)
    word_list = ['identification', 'composition', 'ingredients', 'information', 'measures', 'handling', 'consideration',
                 'properties', 'stability', 'considerations', 'exposure', 'section']
    
    starts_with_section = False
    starts_with_number = False
    splitted_sentence = sentence.split()
    if len(splitted_sentence) > 1:
        starts_with_section = sentence.split()[0] == 'section'
        starts_with_number = sentence[0].isdigit() and not any(char.isdigit() for char in sentence[4:])
        short_sentence = len(sentence) <= 50
    
    contains_title_word = any(word in splitted_sentence for word in word_list) 
    
    if (starts_with_number or starts_with_section) and contains_title_word and short_sentence: 
        return 1
    else:
        return 0

In [3]:
def df_from_text(path):
    text = tx.process(path)
    text = text.decode('utf-8')
    text_splitted = text.split("\r\n")
    
    d = {'sentence': text_splitted}
    df_text = pd.DataFrame(data=d)
    df_text['title'] = 0
    
    df_text["sentence"] = df_text["sentence"].str.lower()
    df_text['title'] = df_text['sentence'].map(lambda a: check_if_title(a))
    return df_text

In [4]:
# voorbeeld
path = './data/10N_Sodium_Hydroxide_NaOH_40_6_US_EN_sds.pdf'
df_text = df_from_text(path)

In [5]:
# select only titles
only_titles = df_text[df_text['title']==1]
only_titles

,sentence,title
12,1 identification,1
32,2 hazard(s) identification,1
135,3 composition/information on ingredients,1
154,4 first-aid measures,1
192,5 fire-fighting measures,1
203,6 accidental release measures,1
247,7 handling and storage,1
261,8 exposure controls/personal protection,1
322,9 physical and chemical properties,1
435,10 stability and reactivity,1


In [6]:
def make_pdf_dict(directory):
    pdf_dict = {}
    key_dict = {}
    for i, filename in enumerate(os.listdir(directory)):
        if filename.endswith(".pdf"):
            file = os.path.join(directory, filename)
            pdf_dict[i] = df_from_text(file)
            key_dict[i] = filename
        
        else:
            print("No PDF files found")
            
    return pdf_dict, key_dict

In [7]:
directory = r'./data'
pdf_dict, key_dict = make_pdf_dict(directory)

for dict_item in pdf_dict.items():
    df = dict_item[1]
    pdf_key = dict_item[0]
    pdf_title = key_dict[pdf_key]
    
    # select only titles
    only_titles = df[df['title']==1]
    
    print(f'PDF: {pdf_key}; title: {pdf_title}: titles: {only_titles.shape[0]}')
    display(only_titles)


PDF: 0; title: 10N_Sodium_Hydroxide_NaOH_40_6_US_EN_sds.pdf: titles: 16


,sentence,title
12,1 identification,1
32,2 hazard(s) identification,1
135,3 composition/information on ingredients,1
154,4 first-aid measures,1
192,5 fire-fighting measures,1
203,6 accidental release measures,1
247,7 handling and storage,1
261,8 exposure controls/personal protection,1
322,9 physical and chemical properties,1
435,10 stability and reactivity,1


PDF: 1; title: 1799516.pdf: titles: 27


,sentence,title
6,1. product and company identification,1
32,2. hazards identification,1
88,3. composition / information on ingredients,1
135,5. fire fighting measures,1
173,6. accidental release measures,1
188,7. handling and storage,1
264,9. physical and chemical properties,1
301,10. stability and reactivity,1
367,12. ecological information,1
372,13. disposal considerations,1


PDF: 2; title: 23114.pdf: titles: 16


...

,sentence,title
3,section 1. identification,1
38,section 2. hazards identification,1
106,section 2. hazards identification,1
114,section 3. composition/information on ingredients,1
152,section 4. first aid measures,1
211,section 4. first aid measures,1
256,section 5. fire-fighting measures,1
306,section 6. accidental release measures,1
340,section 13 for waste disposal.,1
342,section 7. handling and storage,1


PDF: 18; title: Trifluoroacetic_Acid_TFA_7_US_EN_sds.pdf: titles: 16


,sentence,title
12,1 identification,1
38,2 hazard(s) identification,1
180,3 composition/information on ingredients,1
189,4 first-aid measures,1
213,5 fire-fighting measures,1
241,6 accidental release measures,1
266,7 handling and storage,1
280,8 exposure controls/personal protection,1
343,9 physical and chemical properties,1
448,10 stability and reactivity,1


PDF: 19; title: 巴斯夫紫外线吸收剂-tinuvin-1130-msds宝旭化工.pdf: titles: 16


,sentence,title
10,1. identification,1
34,2. hazards identification,1
108,3. composition/information on ingredients,1
153,4. first-aid measures,1
174,5. fire-fighting measures,1
203,6. accidental release measures,1
216,7. handling and storage,1
229,8. exposure controls/personal protection,1
272,9. physical and chemical properties,1
402,10. stability and reactivity,1


In [8]:
df = pdf_dict[7]
only_titles = df[df['title']==1]
len(only_titles.sentence.iloc[8].split())

5

In [9]:
len("3 composition/information on ingredients")

40